In [103]:
import pandas as pd
import glob, os

In [104]:
df = pd.read_csv('../datasets/flights_2021_all.csv')
df_w = pd.read_csv('../datasets/weather_data/ATL_2021.csv',parse_dates=['date'])

In [105]:
df.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operating_Airline,OriginAirportID,...,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,TaxiIn,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,DistanceGroup
0,0,2021,1,1,5,2,2021-01-05,DL,9E,11298,...,1625,1623.0,0.0,0.0,10.0,2005,2000.0,0.0,0.0,4
1,1,2021,1,1,6,3,2021-01-06,DL,9E,11298,...,1625,1628.0,3.0,0.0,11.0,2005,2005.0,0.0,0.0,4
2,2,2021,1,1,12,2,2021-01-12,DL,9E,11298,...,1625,1620.0,0.0,0.0,13.0,2005,2000.0,0.0,0.0,4
3,3,2021,1,1,13,3,2021-01-13,DL,9E,11298,...,1625,1621.0,0.0,0.0,8.0,2005,1954.0,0.0,0.0,4
4,4,2021,1,1,19,2,2021-01-19,DL,9E,11298,...,1625,1625.0,0.0,0.0,9.0,2005,1944.0,0.0,0.0,4


# Datetime Conversion

- A new column <code>CombinedDateTime</code> by combining <code>FlightDate</code> column and   <code>CRSDepTime</code> column.
- Scale the column <code>ScaledCRSDepTime</code> to the nearest hour.

In [106]:
df['CRSDepTime'] = df['CRSDepTime'].astype('str')

In [107]:
df['CombinedDateTime'] = pd.to_datetime(df['FlightDate'] + ' ' + df['CRSDepTime'].str.zfill(4), format='%Y-%m-%d %H%M')

In [108]:
df['ScaledCRSDepTime'] = pd.to_datetime(df['CombinedDateTime']).dt.round('H')


In [109]:
df.sample(10)

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operating_Airline,OriginAirportID,...,DepDelayMinutes,DepDel15,TaxiIn,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,DistanceGroup,CombinedDateTime,ScaledCRSDepTime
333735,333735,2021,3,8,14,6,2021-08-14,DL,DL,11298,...,3.0,0.0,6.0,930,915.0,0.0,0.0,4,2021-08-14 05:49:00,2021-08-14 06:00:00
239571,239571,2021,2,6,8,2,2021-06-08,UA,UA,14771,...,2.0,0.0,13.0,2032,2048.0,16.0,1.0,2,2021-06-08 19:05:00,2021-06-08 19:00:00
443364,443364,2021,4,10,16,6,2021-10-16,UA,UA,13930,...,0.0,0.0,8.0,1242,1230.0,0.0,0.0,4,2021-10-16 10:56:00,2021-10-16 11:00:00
199231,199231,2021,2,6,23,3,2021-06-23,AA,YX,11433,...,0.0,0.0,16.0,1430,1402.0,0.0,0.0,1,2021-06-23 13:52:00,2021-06-23 14:00:00
336362,336362,2021,3,8,12,4,2021-08-12,WN,WN,13232,...,43.0,1.0,6.0,1540,1604.0,24.0,1.0,4,2021-08-12 14:05:00,2021-08-12 14:00:00
302800,302800,2021,3,8,30,1,2021-08-30,AS,OO,14771,...,0.0,0.0,8.0,2358,2336.0,0.0,0.0,2,2021-08-30 22:30:00,2021-08-30 22:00:00
342651,342651,2021,3,8,8,7,2021-08-08,UA,YX,13930,...,4.0,0.0,6.0,1143,1133.0,0.0,0.0,4,2021-08-08 09:05:00,2021-08-08 09:00:00
326250,326250,2021,3,8,1,7,2021-08-01,AA,AA,11298,...,16.0,1.0,3.0,1101,1100.0,0.0,0.0,5,2021-08-01 07:00:00,2021-08-01 07:00:00
175505,175505,2021,2,5,28,5,2021-05-28,WN,WN,12892,...,0.0,0.0,3.0,730,718.0,0.0,0.0,2,2021-05-28 06:00:00,2021-05-28 06:00:00
334529,334529,2021,3,8,11,3,2021-08-11,WN,WN,10821,...,0.0,0.0,4.0,815,757.0,0.0,0.0,6,2021-08-11 06:25:00,2021-08-11 06:00:00


# Weather data

In [110]:
csv_folder = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/'
for file_name in os.listdir(csv_folder):
    print(file_name)
    if file_name!='test':
        dff = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/'+file_name)
        dff.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)
        dff['Origin'] = file_name[:3]
        dff.to_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/test/'+file_name)

SEA_2021.csv
SLC_2021.csv
SFO_2021.csv
ORD_2021.csv
test
ATL_2021.csv
CLT_2021.csv
DEN_2021.csv
LAX_2021.csv
PHX_2021.csv
DTW_2021.csv
PHL_2021.csv
BWI_2021.csv
IAH_2021.csv
MDW_2021.csv
DFW_2021.csv


In [111]:
csv_folder = '../datasets/weather_data/test/'
concatenated_data = pd.DataFrame()
for file_name in os.listdir(csv_folder):
    print(file_name)
    if file_name!='.DS_Store':
        df_ = pd.read_csv(csv_folder+file_name)
        concatenated_data = pd.concat([concatenated_data, df_], ignore_index=True)
concatenated_data.to_csv('../datasets/weather_merged.csv')


SEA_2021.csv
.DS_Store
SLC_2021.csv
SFO_2021.csv
ORD_2021.csv
ATL_2021.csv
CLT_2021.csv
DEN_2021.csv
LAX_2021.csv
PHX_2021.csv
DTW_2021.csv
PHL_2021.csv
BWI_2021.csv
IAH_2021.csv
MDW_2021.csv
DFW_2021.csv


In [112]:
concatenated_data.columns

Index(['Unnamed: 0', 'ScaledCRSDepTime', 'temperature_2m',
       'relative_humidity_2m', 'dew_point_2m', 'precipitation', 'rain',
       'snowfall', 'weather_code', 'surface_pressure', 'cloud_cover',
       'wind_speed_10m', 'wind_direction_10m', 'wind_direction_100m',
       'Origin'],
      dtype='object')

In [113]:
df_w = pd.read_csv('../datasets/weather_merged.csv',parse_dates=['ScaledCRSDepTime'])


In [114]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131400 entries, 0 to 131399
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Unnamed: 0.1          131400 non-null  int64         
 1   Unnamed: 0            131400 non-null  int64         
 2   ScaledCRSDepTime      131400 non-null  datetime64[ns]
 3   temperature_2m        131400 non-null  float64       
 4   relative_humidity_2m  131400 non-null  float64       
 5   dew_point_2m          131400 non-null  float64       
 6   precipitation         131400 non-null  float64       
 7   rain                  131400 non-null  float64       
 8   snowfall              131400 non-null  float64       
 9   weather_code          131400 non-null  float64       
 10  surface_pressure      131400 non-null  float64       
 11  cloud_cover           131400 non-null  float64       
 12  wind_speed_10m        131400 non-null  float64       
 13 

In [115]:
df_w.head()

,Unnamed: 0.1,Unnamed: 0,ScaledCRSDepTime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m,wind_direction_100m,Origin
0,0,0,2021-01-01 00:00:00,7.7745,92.752815,6.6745,0.0,0.0,0.0,3.0,1005.58594,100.0,15.273505,188.13002,194.58896,SEA
1,1,1,2021-01-01 01:00:00,7.7245,91.798290,6.4745,0.0,0.0,0.0,3.0,1005.68200,100.0,12.371645,188.36580,192.72435,SEA
2,2,2,2021-01-01 02:00:00,7.3245,92.408510,6.1745,0.0,0.0,0.0,3.0,1005.85860,100.0,11.212135,174.47255,181.81827,SEA
3,3,3,2021-01-01 03:00:00,7.4245,90.208900,5.9245,0.0,0.0,0.0,3.0,1005.56824,100.0,11.966953,173.08887,180.88138,SEA
4,4,4,2021-01-01 04:00:00,7.5245,88.977980,5.8245,0.0,0.0,0.0,3.0,1005.57330,100.0,12.245294,178.31535,184.26780,SEA


In [116]:
df_w.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)

In [117]:
df_w.head()

,Unnamed: 0.1,Unnamed: 0,ScaledCRSDepTime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m,wind_direction_100m,Origin
0,0,0,2021-01-01 00:00:00,7.7745,92.752815,6.6745,0.0,0.0,0.0,3.0,1005.58594,100.0,15.273505,188.13002,194.58896,SEA
1,1,1,2021-01-01 01:00:00,7.7245,91.798290,6.4745,0.0,0.0,0.0,3.0,1005.68200,100.0,12.371645,188.36580,192.72435,SEA
2,2,2,2021-01-01 02:00:00,7.3245,92.408510,6.1745,0.0,0.0,0.0,3.0,1005.85860,100.0,11.212135,174.47255,181.81827,SEA
3,3,3,2021-01-01 03:00:00,7.4245,90.208900,5.9245,0.0,0.0,0.0,3.0,1005.56824,100.0,11.966953,173.08887,180.88138,SEA
4,4,4,2021-01-01 04:00:00,7.5245,88.977980,5.8245,0.0,0.0,0.0,3.0,1005.57330,100.0,12.245294,178.31535,184.26780,SEA


# Combine two datasets

In [118]:
df.columns 

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'FlightDate', 'Marketing_Airline_Network', 'Operating_Airline ',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'TaxiIn', 'CRSArrTime',
       'ArrTime', 'ArrDelayMinutes', 'ArrDel15', 'DistanceGroup',
       'CombinedDateTime', 'ScaledCRSDepTime'],
      dtype='object')

In [119]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131400 entries, 0 to 131399
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Unnamed: 0.1          131400 non-null  int64         
 1   Unnamed: 0            131400 non-null  int64         
 2   ScaledCRSDepTime      131400 non-null  datetime64[ns]
 3   temperature_2m        131400 non-null  float64       
 4   relative_humidity_2m  131400 non-null  float64       
 5   dew_point_2m          131400 non-null  float64       
 6   precipitation         131400 non-null  float64       
 7   rain                  131400 non-null  float64       
 8   snowfall              131400 non-null  float64       
 9   weather_code          131400 non-null  float64       
 10  surface_pressure      131400 non-null  float64       
 11  cloud_cover           131400 non-null  float64       
 12  wind_speed_10m        131400 non-null  float64       
 13 

In [120]:
merged_df = pd.merge(df, df_w, on=['Origin', 'ScaledCRSDepTime'], suffixes=('', '_df2'))


In [121]:
# merge = pd.merge(df,df_w,on=['Origin','ScaledCRSDepTime'],how='left')

In [122]:
merged_df.columns

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'FlightDate', 'Marketing_Airline_Network', 'Operating_Airline ',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'TaxiIn', 'CRSArrTime',
       'ArrTime', 'ArrDelayMinutes', 'ArrDel15', 'DistanceGroup',
       'CombinedDateTime', 'ScaledCRSDepTime', 'Unnamed: 0.1',
       'Unnamed: 0_df2', 'temperature_2m', 'relative_humidity_2m',
       'dew_point_2m', 'precipitation', 'rain', 'snowfall', 'weather_code',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'wind_direction_100m'],
      dtype='object')

In [123]:
remove_cols = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 0_df2']
merged_df.drop(remove_cols, axis=1, inplace=True)
merged_df.shape

(538040, 36)

In [124]:
merged_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Marketing_Airline_Network', 'Operating_Airline ', 'OriginAirportID',
       'Origin', 'DestAirportID', 'Dest', 'CRSDepTime', 'DepTime',
       'DepDelayMinutes', 'DepDel15', 'TaxiIn', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'DistanceGroup', 'CombinedDateTime',
       'ScaledCRSDepTime', 'temperature_2m', 'relative_humidity_2m',
       'dew_point_2m', 'precipitation', 'rain', 'snowfall', 'weather_code',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'wind_direction_100m'],
      dtype='object')

In [125]:
merged_df.to_csv('../datasets/final_dataset.csv')

# Summary

1. First the weather data for all the major airport cities from directory <code>/datasets/weather_data</code> is combined and a file <code>/datasets/weather_merged.csv</code> is created. 
2. The file generated at 1 and the file received from eda-1 is merged on <code>ScaledCRSDepTime</code> and <code>Origin</code> columns. The final output of the file is stored in directory <code>/datasets/Flights_2021/final_dataset-add.csv</code>. The file is not added in this repo because of the size. Press this [button](https://drive.google.com/file/d/13H98BcH-7ojUKEEZRDzSrAx4Oq5J_Gp6/view?usp=sharing) to download the dataset. (Note: You need to be logged in with your ait email to access the data.)